# Code to generate synthetic commit data
Wrote this before we had the commit chronicles dataset, you can disregard this step and jump directly to the next, or you can use it as a preliminary testing area

In [ ]:
import itertools
import json
import random

import numpy as np
import pandas as pd
import rootutils
import torch.nn as nn
from tokenizers import Tokenizer, decoders, models, pre_tokenizers, processors, trainers
from torch.utils.data import DataLoader, Dataset

In [ ]:
ROOT = rootutils.setup_root(".", ".project-root", pythonpath=True)
OUTPUT_DIR = ROOT / "data/playground"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
function_names = [
    "display_greeting",
    "show_warning",
    "print_info",
    "log_event",
    "announce_start",
    "notify_completion",
    "output_status",
    "send_alert",
    # "record_activity", "trace_execution", "emit_signal", "broadcast_message",
    # "report_error", "inform_user", "update_display", "refresh_view",
    # "render_output", "present_cdata", "show_notification", "display_result",
    # "print_summary", "log_details", "notify_admin", "output_log",
    # "send_update", "record_log", "trace_process", "emit_event", "broadcast_alert",
    # "report_status", "inform_system",
    # "initialize_module", "shutdown_service", "handle_request", "process_data",
    # "validate_input", "authenticate_user", "authorize_access", "compress_files",
    # "decompress_files", "backup_database", "restore_database", "sync_files",
    # "monitor_performance", "optimize_queries", "generate_report", "send_email",
    # "schedule_task", "cancel_task", "retry_operation", "log_transaction",
    # "update_configuration", "load_settings", "parse_response", "manage_sessions",
    # "encrypt_data", "decrypt_data", "format_output", "validate_credentials",
    # "handle_errors", "process_payments", "manage_notifications", "track_usage",
    # "generate_token", "verify_identity", "log_out", "register_user"
]

printed_messages = [
    "Hello, World!",
    "Warning: Low Disk Space.",
    "Information: Process started.",
    "Event logged successfully.",
    "Starting the application.",
    "Completion successful.",
    "Status: All systems operational.",
    "Alert: Unauthorized access detected.",
    # "Activity recorded.", "Execution trace started.", "Signal emitted.",
    # "Message broadcasted.", "Error encountered in module.", "User informed.",
    # "Display updated.", "View refreshed.", "Output rendered.", "Data presented.",
    # "Notification displayed.", "Result shown.", "Summary printed.",
    # "Details logged.", "Admin notified.", "Log output generated.",
    # "Update sent to server.", "Log recorded.", "Process traced.", "Event emitted.",
    # "Alert broadcasted.", "Status reported.", "System informed.",
    # "Module initialized.", "Service shutdown gracefully.", "Request handled successfully.",
    # "Data processed without errors.", "Input validated.", "User authenticated.",
    # "Access authorized.", "Files compressed.", "Files decompressed.",
    # "Database backed up.", "Database restored.", "Files synchronized.",
    # "Performance monitored.", "Queries optimized.", "Report generated.",
    # "Email sent successfully.", "Task scheduled.", "Task canceled.",
    # "Operation retried.", "Transaction logged.", "Configuration updated.",
    # "Settings loaded.", "Response parsed.", "Session managed.",
    # "Data encrypted.", "Data decrypted.", "Output formatted.", "Credentials validated.",
    # "Errors handled.", "Payments processed.", "Notifications managed.",
    # "Usage tracked.", "Token generated.", "Identity verified.", "User logged out.",
    # "User registered successfully."
]

file_names = [
    "utils.py",
    "helpers.py",
    "main.py",
    "scripts.py",
    "commands.py",
    "logger.py",
    "notifications.py",
    "functions.py",
    "actions.py",
    "alerts.py",
    "outputs.py",
    "interactions.py",
    "communication.py",
    # "handlers.py", "response.py", "initializer.py", "setup.py", "runner.py",
    # "manager.py", "processor.py", "controller.py", "service.py", "adapter.py",
    # "connector.py", "dispatcher.py", "executor.py", "facade.py", "gateway.py",
    # "handler.py", "integrator.py", "mediator.py", "observer.py", "provider.py",
    # "registrar.py", "scheduler.py", "translator.py", "validator.py", "watcher.py",
    # "database.py", "authentication.py", "authorization.py", "backup.py",
    # "monitor.py", "reporting.py", "email_service.py", "task_manager.py",
    # "transaction.py", "compression.py", "decompression.py", "synchronization.py",
    # "performance.py", "optimization.py",
    # "config.py", "utils_v2.py", "helpers_v2.py", "main_v2.py",
    # "scripts_v2.py", "commands_v2.py", "logger_v2.py", "notifications_v2.py",
    # "functions_v2.py", "actions_v2.py", "alerts_v2.py", "outputs_v2.py",
    # "interactions_v2.py", "communication_v2.py", "handlers_v2.py",
    # "response_v2.py", "initializer_v2.py", "setup_v2.py", "runner_v2.py",
    # "manager_v2.py", "processor_v2.py", "controller_v2.py", "service_v2.py",
    # "adapter_v2.py", "connector_v2.py", "dispatcher_v2.py", "executor_v2.py",
    # "facade_v2.py", "gateway_v2.py", "handler_v2.py", "integrator_v2.py",
    # "mediator_v2.py", "observer_v2.py", "provider_v2.py", "registrar_v2.py",
    # "scheduler_v2.py", "translator_v2.py", "validator_v2.py", "watcher_v2.py",
    # "database_v2.py", "authentication_v2.py", "authorization_v2.py",
    # "backup_v2.py", "monitor_v2.py", "reporting_v2.py", "email_service_v2.py",
    # "task_manager_v2.py", "transaction_v2.py", "compression_v2.py",
    # "decompression_v2.py", "synchronization_v2.py", "performance_v2.py",
    # "optimization_v2.py"
]


def generate_file_content(function_name, message):
    """
    Generates the complete content of a Python file by concatenating
    the new function definition.
    """
    function_def = f"def {function_name}():\n" f'    print("{message}")\n'
    return function_def


def generate_commit_message(message):
    return f'Added function to print "{message}"'


def generate_all_combinations():
    dataset = []
    file_version = {file_name: 0 for file_name in file_names}  # Initialize version tracking

    for file_name, function_name, message in itertools.product(
        file_names, function_names, printed_messages
    ):
        file_version[file_name] += 1
        version = file_version[file_name]

        file_content = generate_file_content(function_name, message)

        commit_message = generate_commit_message(message)

        dataset.append(
            {
                "file_name": file_name,
                "version": version,
                "commit_diff": file_content,
                "commit_message": commit_message,
            }
        )

    return dataset


print("Generating all possible combinations. This may take a while...")
dataset = generate_all_combinations()
print(f"Total combinations generated: {len(dataset)}")
with open(OUTPUT_DIR / "commit_dataset.json", "w") as f:
    json.dump(dataset, f, indent=2)
print("Dataset generated and saved to commit_dataset.json")
df = pd.DataFrame(dataset)

In [ ]:
import torch.utils.data
from datasets import load_from_disk

from src.data.types import SingleExample

dataset = load_from_disk(OUTPUT_DIR / "02-processed-validation")
dataset.select(range(10)).to_pandas()

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using device: {device}")


commit_diffs = df["commit_diff"].tolist() + df["commit_message"].tolist()
tokenizer = Tokenizer(models.BPE())

tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
tokenizer.decoder = decoders.ByteLevel()

special_tokens = ["<pad>", "<s>", "</s>", "<unk>", "<mask>"]

trainer = trainers.BpeTrainer(vocab_size=3000, special_tokens=special_tokens)

tokenizer.train_from_iterator(commit_diffs, trainer=trainer)

tokenizer.post_processor = processors.TemplateProcessing(
    single="<s> $A </s>",
    pair="<s> $A </s> </s> $B </s>",
    special_tokens=[
        ("<s>", tokenizer.token_to_id("<s>")),
        ("</s>", tokenizer.token_to_id("</s>")),
    ],
)

tokenizer.save(str(OUTPUT_DIR / "bpe_tokenizer.json"))

tokenizer = Tokenizer.from_file(str(OUTPUT_DIR / "bpe_tokenizer.json"))


def encode_text(text, max_length):
    encoding = tokenizer.encode(text)
    if len(encoding.ids) > max_length:
        encoding = encoding.slice(0, max_length)
    else:
        encoding.pad(length=max_length)
    return encoding.ids


max_input_length = 100
max_target_length = 100

input_ids = [encode_text(diff, max_input_length) for diff in df["commit_diff"].tolist()]
target_ids = [encode_text(msg, max_target_length) for msg in df["commit_message"].tolist()]

input_ids = torch.tensor(input_ids)
target_ids = torch.tensor(target_ids)

dataset_size = len(df)
indices = list(range(dataset_size))
split = int(np.floor(0.2 * dataset_size))
random.seed(42)
random.shuffle(indices)
train_indices, test_indices = indices[split:], indices[:split]

train_inputs, train_targets = input_ids[train_indices], target_ids[train_indices]
test_inputs, test_targets = input_ids[test_indices], target_ids[test_indices]

print(f"Training samples: {len(train_inputs)}")
print(f"Testing samples: {len(test_inputs)}")


class TransformerModel(nn.Module):
    def __init__(self, vocab_size, embed_size, num_heads, hidden_dim, num_layers, dropout=0.1):
        super(TransformerModel, self).__init__()
        self.embed_size = embed_size
        self.token_embedding = nn.Embedding(vocab_size, embed_size)
        self.positional_encoding = nn.Parameter(
            self._generate_positional_encoding(5000, embed_size), requires_grad=False
        )
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_size, nhead=num_heads, dim_feedforward=hidden_dim, dropout=dropout
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(embed_size, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def _generate_positional_encoding(self, max_len, embed_size):
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_size, 2) * (-np.log(10000.0) / embed_size))
        pe = torch.zeros(max_len, embed_size)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        return pe

    def forward(self, src, src_mask=None):
        src = self.token_embedding(src) + self.positional_encoding[: src.size(1), :]
        src = self.dropout(src)
        memory = self.transformer_encoder(src.permute(1, 0, 2), src_mask)
        output = self.fc_out(memory.permute(1, 0, 2))
        return output


vocab_size = tokenizer.get_vocab_size()
embed_size = 256
num_heads = 8
hidden_dim = 512
num_layers = 3
dropout = 0.1

model = TransformerModel(vocab_size, embed_size, num_heads, hidden_dim, num_layers, dropout).to(
    device
)


class CommitDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return {"input_ids": self.inputs[idx], "target_ids": self.targets[idx]}


batch_size = 2

train_dataset = CommitDataset(train_inputs, train_targets)
test_dataset = CommitDataset(test_inputs, test_targets)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

pad_token_id = tokenizer.token_to_id("<pad>")
criterion = nn.CrossEntropyLoss(ignore_index=pad_token_id)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


def train(model, loader, optimizer, criterion, device):
    model.train()
    epoch_loss = 0
    for batch in loader:
        input_ids = batch["input_ids"].to(device)
        target_ids = batch["target_ids"].to(device)

        optimizer.zero_grad()
        output = model(input_ids, src_mask=None)

        output = output.contiguous().view(-1, vocab_size)  # (batch_size * seq_len, vocab_size)
        target = target_ids.contiguous().view(-1)  # (batch_size * seq_len)

        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(loader)


def evaluate(model, loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for batch in loader:
            input_ids = batch["input_ids"].to(device)
            target_ids = batch["target_ids"].to(device)

            output = model(input_ids, src_mask=None)

            output = output.contiguous().view(-1, vocab_size)  # (batch_size * seq_len, vocab_size)
            target = target_ids.contiguous().view(-1)  # (batch_size * seq_len)

            loss = criterion(output, target)
            epoch_loss += loss.item()
    return epoch_loss / len(loader)


# Training Loop
num_epochs = 10

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer, criterion, device)
    val_loss = evaluate(model, test_loader, criterion, device)
    print(
        f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}"
    )


def generate_commit_message_(model, tokenizer, commit_diff, max_length=64):
    model.eval()
    with torch.no_grad():
        encoding = tokenizer.encode(commit_diff)
        if len(encoding.ids) > max_input_length:
            encoding = encoding.slice(0, max_input_length)
        else:
            encoding.pad(length=max_length)  # Pad in place
        input_ids = torch.tensor([encoding.ids]).to(device)

        outputs = model(input_ids)
        predicted_ids = outputs.argmax(dim=-1).squeeze().cpu().numpy()

        decoded = tokenizer.decode(predicted_ids)
        decoded = decoded.replace("<s>", "").replace("</s>", "").strip()
        return decoded


new_commit_diff = '''def list_users():
    """Lists all users in the system."""
    return database.find_all()
'''

generated_message = generate_commit_message_(model, tokenizer, new_commit_diff)
print(f"Commit Diff:\n{new_commit_diff}\n")
print(f"Generated Commit Message: {generated_message}")